In [ ]:
import random
import pandas as pd
import scipy.stats
import numpy as np
import scipy.stats

In [ ]:
def auction(rule, user_bid, bids):
    print(bids, type(bids))
    print(user_bid, type(user_bid))
    
    # Introduce heterogeneous "objective" values for each auction
    # Vary agents' bids as a percent of each "objective" value
    glob_value = random.randint(3000,10000)
    user_bid_adj = (round(user_bid[0] * glob_value /100), user_bid[1])
    bids_adj = []
    for bid in bids:
        bids_adj = bids_adj + [((round(bid[0] * glob_value /100), bid[1]))]
        
    # Add user bid to agents bids   
    final_bid_list = bids_adj + [(user_bid_adj)]
    print(type(final_bid_list))
    print(final_bid_list)
    
    # Rank bids from highest to lowest
    bid_rank = sorted(final_bid_list, key = lambda tup: tup[0], reverse=True)
    print("bid rank")
    print(bid_rank)
    
    # Develop auction rules
    
    # Vickrey Second-Price Auction
    if rule == 'second price':
        bidder = bid_rank[0][1]
        price = bid_rank[1][0]
        
    # Percentage based rule, unless 2nd price greater than outcome
    elif type(rule) == int:
        bidder = bid_rank[0][1]
        if bid_rank[1][0] > (rule * bid_rank[0][0]):
            price = bid_rank[1][0]
        else:
            price = round(bid_rank[0][0] * (float(rule)) /100)
            
        
        
    # Sets a soft floor reserve price, under which it acts as a first price auction
    elif rule == "soft floor":
        s = float(random.randint(93,98))
        if bid_rank[1][0]  < (glob_value * s)/100 :
            bidder = bid_rank[0][1]
            price = bid_rank[0][0]
        
        else:
            bidder = bid_rank[0][1]
            price = bid_rank[1][0]
            
    # First price rule for use in mixed strategy rules        
    elif rule == "first price":
        bidder = bid_rank[0][1]
        price = bid_rank[0][0]
    
    # Auction results
    if bidder == user_bid[1]:
        user_value = round(user_bid_adj[0]/0.9)
        return (1, user_value - price , (user_value - price)/user_value)
    else:
        return (0, 0, 0)



In [ ]:
def user_bid(user_value, user_name):
    return(tuple((user_value, user_name)))

In [ ]:
def user_value():
    value = float(np.random.normal(loc = 75, scale = 20, size = 1 ))
    return value * 0.9




In [ ]:
#introduce heterogeneity in both the number of bidders and their bids
def competitor_bids():
    bids = []
    choices = [5,6,6,7,7,8,8,8,9,9,9,9,10,10,10,11,11,12,12,13]
    for i in range(0, random.choice(choices)):
        bids.append((float(np.random.normal(loc = 75 , scale = 40 , size = 1)), i))
    return bids

In [ ]:
second_price_log = []
for i in range(0,72):
    second_price = [["second price"], []]
    for j in range(0,1000):
        second_price[1] = second_price[1] + [(auction('second price', user_bid(user_value(), 501), competitor_bids()))]
    second_price_log.append(second_price)
#print(response_log)

constant_percent_log = []
for i in range(0,18):
    constant_percent = [["constant percent"], []]
    r = random.randint(90, 95)
    for j in range(0,1000):
        constant_percent[1] = constant_percent[1] + [(auction(r, user_bid(user_value(), 501), competitor_bids()))]
    constant_percent_log.append(constant_percent)
#print(dishonest_response_log)
variable_percent_log = []
for i in range(0,18):
    variable_percent = [["variable percent"], []]
    random_percents = []
    for j in range(0,1000):
        random_percents = random_percents + [(random.randint(90,95))]
        variable_percent[1] = variable_percent[1] + [(auction(random_percents[j], user_bid(user_value(), 501), competitor_bids()))]
    variable_percent_log.append(variable_percent)
#print(variable_per_response_log)

soft_floor_log = []
for i in range(0,18):
    soft_floor = [["soft floor"], []]
    for j in range(0,1000):
        soft_floor[1] = soft_floor[1] + [(auction("soft floor", user_bid(user_value(), 501), competitor_bids()))]
    soft_floor_log.append(soft_floor)
#print(variable_per_response_log)

mixed_strategy_log = []
for i in range(0,18):
    mixed_strategy = [["mixed strategy"], []]
    for j in range(0,1000):
        r = random.randint(93,98)
        strat_list = ["second price", "soft floor", "first price", r]
        mixed_strategy[1] = mixed_strategy[1] + [(auction(random.choice(strat_list), user_bid(user_value(), 501), competitor_bids()))]
    mixed_strategy_log.append(mixed_strategy)
#print(variable_per_response_log)




In [ ]:
log_list = [second_price_log, constant_percent_log, 
            variable_percent_log, soft_floor_log, mixed_strategy_log]

listrows = []
for log in log_list:
    for sublog in log:
        returns = []
        returns_per = []
        for entry in sublog[1]:
            if entry[0] != 0:
                returns.append(entry[1])
                returns_per.append(entry[2])
        df_return = pd.DataFrame()
        df_per_returns = pd.DataFrame()
        df_per_returns['returns'] = returns_per
        df_return['returns'] = returns
    #df_return['percentage returns'] = returns_per
        returns_list = sublog[0]
        returns_list.extend(list(df_return['returns'].describe()))
        returns_list.extend(list(df_per_returns['returns'].describe()))
        returns_list.extend([df_per_returns['returns'].skew(), df_per_returns['returns'].kurtosis()])
        returns_list.append(scipy.stats.moment(df_per_returns["returns"], 4))
        returns_list.append(scipy.stats.moment(df_per_returns["returns"], 5))
        listrows.append(returns_list)
    
print(listrows[0])   


In [ ]:
def addRow(df,ls):
    """
    Given a dataframe and a list, append the list as a new row to the dataframe.

    :param df: <DataFrame> The original dataframe
    :param ls: <list> The new row to be added
    :return: <DataFrame> The dataframe with the newly appended row
    """

    numEl = len(ls)

    newRow = pd.DataFrame(np.array(ls).reshape(1,numEl), columns = list(df.columns))

    df = df.append(newRow, ignore_index=True)

    return df

In [ ]:
df_rows = pd.DataFrame(columns = ['auction type', 'count' , 'mean', 'std', 'min', '25', '50', '75', 'max' , 
                  'count_per', 'mean_per', 'std_per', 'min_per', '25_per', '50_per', '75_per',
                   'max_per', 'skew', 'kurtosis', '4th', '5th'])
for row in listrows:
    df_rows = addRow(df_rows, row)
df_rows.info()

In [ ]:
df_rows.to_csv("data_exp_1_2.csv", sep = ",")